# X-ray diffraction (XRD) spectra plotting

*Authors: Enze Chen (University of California, Berkeley)*

![Powder XRD spectra](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/XRD_labeled.png)

This is an interactive notebook that teaches you how to compute the powder XRD spectra from experimental parameters ($a$, $\lambda$, $T$, etc.). In all the times I've learned XRD (which is something like 5 different classes at this point), I have never once seen a discussion for how these various parameters *precisely* result in the spectra we observe using a diffractometer. This is an opportunity for you to do just that with some Python code.

## Prerequisites

To get the most out of this notebook, you should already have:    
- Knowledge of XRD fundamentals such as Bragg's law and intensity factors.
- Knowledge of scientific computing fundamentals to understand the NumPy functions.

## Learning goals

By the end of this notebook, you should be able to:    
- *Identify* all the physical phenomena and quantities that are important for XRD.
- *Write* code to represent the physical quantities in XRD.
- *Generate* a powder XRD spectra for a candidate crystal structure.

## How to use this notebook

If you are viewing this notebook on [Google Colaboratory](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/XRD_trends.ipynb), then everything is already set up for you (hooray). If you want to run the notebook locally, make sure all the Python libraries in the [`requirements.txt`](https://github.com/enze-chen/learning_modules/blob/master/requirements.txt) file are installed.

For pedagogical reasons, there are a few sections for you to complete the code in order to make the final plot. These sections are delineated with the dashed lines as follows, and you should **only change what's inside**. You don't have to edit the text or code anywhere else. I've also included "**TODO**" to separate instructions from the background context.
```python
# ---------------------- #
# YOUR CODE HERE

# ---------------------- #
```
If you edit the code in a cell, just press `Shift+Enter` to run it again. You have to execute **all** the code cells in this notebook from top to bottom (so don't skip around). A number `[#]` will appear to the left of the code cell once it's done executing. When done successfully, you'll be able to generate a simple XRD spectra at the end.

We will reference [*Elements of X-Ray Diffraction* (3rd)](https://www.pearson.com/us/higher-education/program/Cullity-Elements-of-X-Ray-Diffraction-3rd-Edition/PGM113710.html) by Cullity and Stock, which is a fantastic textbook. Important equations and concepts will be addressed as they come up. Let's dive right in!

----------------------------------

## Python library imports

These are all the required Python libraries.

In [ ]:
# General libraries
import itertools

# Scientific computing libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'figure.figsize':(8,6), 'font.size':20, 'axes.linewidth':2,
                     'lines.linewidth':4, 'lines.markersize':10})

## Step 1: Bragg's law

Not surprisingly, we start with Bragg's law, given by 

$$n\lambda = 2d \sin(\theta)$$

where $n$ is the order (assumed to be $1$), $\lambda$ is the wavelength, $d$ is the interplanar spacing, and $\theta$ is the Bragg angle. 

We know $\lambda$ and can calculate $d$ from the crystal structure, so now we need to find $\theta$. Remember: here we *know* our material and want to *find* its powder XRD spectra.

**TODO**: Complete the following function named `compute_theta()`. Some hints are:
- For cubic systems, we can calculate the interplanar spacing with the following equation:    
$$ d = \frac{a}{\sqrt{h^2 + k^2 + l^2}} $$
where $h,k,l$ are the miller indices of the diffracting plane and $a$ is the lattice constant.
- In order to efficiently calculate the denominator of the above equation for many planes, consider the method [`np.linalg.norm(x, axis=1)`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html).
- To divide a scalar by a NumPy array, use [`np.divide(scalar, array)`](https://numpy.org/doc/stable/reference/generated/numpy.divide.html). You can do the same for two NumPy arrays.
- You will have to calculate $\theta$ by inverting Bragg's law. Do this on paper first, then check out the [`np.arcsin(x)`](https://numpy.org/doc/stable/reference/generated/numpy.arcsin.html) method.

In [ ]:
def compute_thetas(planes, a, wavelength):
    """This function returns a NumPy array of Bragg angles
       computed from its inputs.
    
    Args:
        planes (numpy.darray): A 2D NumPy array where each row
            is a list of [h,k,l] indices.
        a (float): Lattice constant in nanometers.
        wavelength (float): X-ray wavelength in nanometers.
    
    Returns:
        thetas (numpy.darray): A NumPy array of Bragg angles in radians.
    """
    
    # ---------------------- #
    # YOUR CODE HERE
    
    thetas = None  # replace this
    
    # ---------------------- #
    
    return thetas

#### Test for `compute_thetas()`

There are many good reasons to break up the calculation into individual functions, one of them being that we can easily combine them all at the end. Another reason is that we can write **some simple tests to check our logic before proceeding further**. This is very good practice! Assuming you don't change my default values below for Ni lattice constant and Cu $K$-$\alpha$, you should get approximately: `[0.220, 0.314, 0.388, 0.452]` in radians.

In [ ]:
# Run this cell to test the above function
a = 0.3524  # Lattice constant of nickel in nm
wavelength = 0.15406  # Cu K-alpha wavelength in nm
planes = np.array([[1,0,0], [1,1,0], [1,1,1], [2,0,0]])
thetas = compute_thetas(planes, a, wavelength) 
print(f'Angles: {thetas}')

--------------------

## Step 2: Intensities

In a way, if you got the previous function to work, then you're done! You can make a simple [stem plot](https://matplotlib.org/3.2.1/gallery/lines_bars_and_markers/stem_plot.html#sphx-glr-gallery-lines-bars-and-markers-stem-plot-py) that indicates which $2\theta$ peaks you should see, but you won't have the right intensities. For that added challenge, recall that the equation for the **Intensity** is given by

$$ I = |F|^2 \times P \times L \times m \times A \times T $$

where
* $F$ is the Structure factor (we take the modulus before squaring because it can be a complex number).
* $P$ is the Polarization factor.
* $L$ is the Lorentz factor.
* $m$ is the Multiplicity factor.
* $A$ is the Absorption factor.
* $T$ is the Temperature factor.

In particular, we'll calculate $F$, $P$, $L$, $m$ explicitly and leave the remaining two as an exercise to the reader.

### 2.1: The structure factor

The structure factor accounts for how the crystal structure and its basis "select" which planes will have a high diffraction intensity and which planes will be suppressed. It is arguably the most important one!

**TODO**: Complete the following function named `compute_F()`. Some hints are:    
- The formula for the structure factor is given by (Ch. 12 of [Structure of Materials](https://www.cambridge.org/core/books/structure-of-materials/22A17D7856B8472E7B73B38F1147C0FD) by De Graef and McHenry):
$$ F_{hkl} = \sum_{j=1}^{N} f_j (s) \cdot \exp \left( 2 \pi i \left( hu_j + kv_j + lw_j \right) \right), \quad s = \frac{\sin \left( \theta_{hkl} \right)}{\lambda} $$

- While we're not typically taught the functional form of the atomic scattering factor $f_j$, De Graef provides it for us as: 
$$ f(s) = Z - 41.78214 \times s^2 \times \sum_{i=1}^{N} a_i \exp \left( -b_i s^2 \right) $$
where $a_i$ and $b_i$ for some common elements are given by:    

| Name | Structure | $Z$ | $a_1$ | $b_1$ | $a_2$ | $b_2$ | $a_3$ | $b_3$ | $a_4$ | $b_4$ |
| :---: | :---: | ---: | ---: | ---: | ---: | ---: | ---: | ---: | ---: | ---: |
| Si | DC | 14 | 2.129 | 57.775 | 2.533 | 16.476 | 0.835 | 2.880 | 0.322 | 0.386 | 
| Fe | BCC | 26 | 2.544 | 64.424 | 2.343 | 14.880 | 1.759 | 2.854 | 0.506 | 0.350 |
| Ni | FCC | 28 | 2.210 | 58.727 | 2.134 | 13.553 | 1.689 | 2.609 | 0.524 | 0.339 |

We'll make this one of the inputs to the function below and give the atomic scattering factor elsewhere.

- I strongly recommend starting off with `for` loops to handle the multiplication and use intermediate variables to store the temporary values.
- If the dimensions are confusing, $f$ and $F$ have the same dimensions. Use this fact to guide you.

In [ ]:
def compute_F(planes, basis, f):
    """This function computes the structure factor.
    
    Args:
        planes (numpy.darray): A 2D NumPy array where each row
            indicates the h,k,l the indices for a plane.
        basis (numpy.darray): A 2D NumPy array where each row
            indicates the coordinates of a basis atom.
        f (numpy.darray): A NumPy array where each element
            is the atomic scattering factor at an angle.
    
    Returns: 
        F (numpy.darray): A NumPy array of the
            structure factor for each angle (diffraction plane).
    """
    
    # ---------------------- #
    # YOUR CODE HERE

    F = None  # replace this
    
    # ---------------------- #
    
    return F

#### Test for `compute_F()`

Again, we'll write a little test for this rather complex function. It's good practice and a good sanity check. Assuming you use my default values for Ni, you should get approximately: `[0, 0, 82.1, 76.8]` corresponding to [FCC selection rules](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/XRD_selection_rules.png).

In [ ]:
# Run this cell to test the above function
s = np.sin(thetas) / (10*wavelength)
S = 2.210 * np.exp(-58.727*s**2) + 2.134 * np.exp(-13.553*s**2) + \
    1.689 * np.exp(-2.609*s**2) + 0.524 * np.exp(-0.339*s**2)
f = 28 - 41.78214 * np.multiply(s**2, S)
basis = np.array([[0,0,0], [0.5,0.5,0], [0.5,0,0.5], [0,0.5,0.5]])
F = compute_F(planes, basis, f)
print(f'Structure factor = {F}')

### 2.2: The multiplicity factor

The multiplicity factor counts the number of planes belonging to one family. Since we're only working with cubic systems in this notebook, this is equivalent to counting the number of permutations and sign flips.

**TODO**: Complete the following function named `compute_m()`. There are many ways to accomplish this. Some hints for what I think is a logical way are:
- Can you separate the computation for number of permutations and the number of different sign combinations?
- You might find the `set()` function combined with [`itertools.permutations()`](https://www.geeksforgeeks.org/python-itertools-permutations/) useful.
- You might find the [`np.count_nonzero(x)`](https://numpy.org/doc/stable/reference/generated/numpy.count_nonzero.html) function helpful for computing the contribution from sign flips.
- You can raise a scalar to an exponent that's a NumPy array with the `scalar**array` syntax.

In [ ]:
def compute_m(planes):
    """This function computes the multiplicity factor for
       each plane in a list of planes.
    
    Args:
        planes (numpy.darray): A 2D NumPy array where each row
            indicates the h,k,l the indices for a plane.
            
    Returns:
        m (numpy.darray): A NumPy array of the 
            multiplicity factor for each plane.
    """
    
    # ---------------------- #
    # YOUR CODE HERE

    m = None  # replace this
    
    # ---------------------- #
    
    return m

#### Test for `compute_m()`

Another function, another test! Using the planes defined previously, you should get: `[6, 12, 8, 6]`.

In [ ]:
# Run this cell to test the above function
planes = np.array([[1,0,0], [1,1,0], [1,1,1], [2,0,0]])
m = compute_m(planes)
print(f'Multiplicity factor: {m}')

### 2.3 The Lorentz-polarization factor

It is common to combine the Lorentz and polarization factors into a single term given by:
$$ L_p(\theta) = \frac{1 + \cos^2 (2\theta)}{\sin^2 (\theta) \cos(\theta)} $$

**TODO**: Complete the following function named `compute_Lp()`. Some hints are:
- You can start with a `for` loop, but to compute $L_p$ efficiently at each angle, you can experiment with built-in NumPy functions such as [`np.divide()`](https://numpy.org/doc/stable/reference/generated/numpy.divide.html), [`np.multiply()`](https://numpy.org/doc/stable/reference/generated/numpy.multiply.html), [`np.sin()`](https://numpy.org/doc/stable/reference/generated/numpy.sin.html), and [`np.cos()`](https://numpy.org/doc/stable/reference/generated/numpy.cos.html).

In [ ]:
def compute_Lp(thetas):
    """This function computes the Lorentz-Polarization factor.
    
    Args:
        thetas (numpy.darray): A NumPy array of floats
            representing angles in radians.
            
    Returns:
        Lp (numpy.darray): A NumPy array of the
            Lorentz-polarization factor at each angle.
    """
    
    # ---------------------- #
    # YOUR CODE HERE

    Lp = None  # replace this
    
    # ---------------------- #
    
    return Lp

#### Test for `compute_Lp()`

Final test! You should get approximately: `[38.99, 18.20, 11.37, 8.04]`.

In [ ]:
# Run this cell to test the above function
planes = np.array([[1,0,0], [1,1,0], [1,1,1], [2,0,0]])
thetas = compute_thetas(planes, a, wavelength) 
Lp = compute_Lp(thetas)
print(f'Lorentz-polarization factor: {Lp}')

## Step 3: Putting it all together

Almost done! First we'll define a few more planes in order to get a few more peaks. Then, we can call the functions we wrote above, whichever ones we were able to get successfully working, to compute the final intensity. 

**TODO**: Compute the final set of `angles` and `intensities` based on your code above. Some hints:
- Don't forget to take the modulus squared of the structure factor!
- To multiply the various intensity terms element-by-element, use the [`np.multiply(x,y)`](https://numpy.org/doc/stable/reference/generated/numpy.multiply.html) method.
- If you didn't get any of the intensity calculations to work, you can just set the `intensities` variable to be [an array of ones](https://numpy.org/doc/stable/reference/generated/numpy.ones.html).
- You'll probably want to convert $\theta$ (radians) into $2\theta$ (degrees) to follow convention. [`np.degrees(x)`](https://numpy.org/doc/stable/reference/generated/numpy.degrees.html) is helpful.
- Here we use a [stem plot](https://matplotlib.org/3.2.1/gallery/lines_bars_and_markers/stem_plot.html#sphx-glr-gallery-lines-bars-and-markers-stem-plot-py) to clearly represent the peaks.

In [ ]:
planes = np.array([[1,0,0], [1,1,0], [1,1,1], [2,0,0], [2,1,0], 
                   [2,1,1], [2,2,0], [2,2,1], [3,0,0], [3,1,0], 
                   [3,1,1], [2,2,2], [3,2,0], [3,2,1], [4,0,0]])
thetas = compute_thetas(planes, a, wavelength) 
s = np.sin(thetas) / (10*wavelength)
S = 2.210 * np.exp(-58.727*s**2) + 2.134 * np.exp(-13.553*s**2) + \
    1.689 * np.exp(-2.609*s**2) + 0.524 * np.exp(-0.339*s**2)
f = 28 - 41.78214 * np.multiply(s**2, S)

# ---------------------- #
# YOUR CODE HERE


# ---------------------- #

fig, ax = plt.subplots()
ax.stem(angles, intensities, basefmt='k--', use_line_collection=True)
ax.set_xlabel(r'$2\theta$ (degree)')
ax.set_ylabel('Intensity (a.u.)')
ax.set_xlim(20, 140)
ax.tick_params(left=False, labelleft=False, direction='in', length=10, width=2)
plt.show()

And I hope you got a pretty stem plot to appear!

--------------------------------------

## Possible extensions

There are *so* many ways you can improve this notebook. Here are just a few ideas:
* [Easy] Does the same code structure work for a different crystal structure, such as body-centered cubic or diamond cubic? I've provided some data above for you to use.
* [Medium] How does **temperature** affect our sample and the resulting XRD spectra? How might you incorporate these effects into the code?
* [Medium] Can you [annotate](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.axes.Axes.annotate.html) the plot so we know which plane corresponds to each peak? This is good practice with [list comprehension](https://www.pythonforbeginners.com/basics/list-comprehensions-in-python) and the snazzy [f-strings](https://docs.python.org/3/reference/lexical_analysis.html#f-strings) syntax.
* [Hard] If you haven't done so already, can you write `compute_F()` in a single return line? Hint: Clever use of matrix multiplication and [`np.sum()`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html). What about the other functions?
* [Hard] Instead of a stem plot, what if you modeled the peaks with [Gaussians](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html)? The really cool part of having a width is the ability to incorporate **thickness effects** through the [Scherrer equation](https://en.wikipedia.org/wiki/Scherrer_equation).

I've implemented a few of the above ideas in [this other demo](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/XRD_trends.ipynb). You can also reference that notebook if you get stuck on any of the above parts.

Also if you want to see the calculated spectra for more structures and compounds, you can find them on [The Materials Project](https://materialsproject.org/).

-----------------------

## Conclusion
I hope you found this notebook helpful in reinforcing your knowledge of XRD and teaching you how to translate those concepts into code for visualization. 
Please don't hesitate to reach out if you have any questions or ideas to contribute.


## Acknowledgements

I thank Laura Armstrong for helpful discussions. 
I also thank [Prof. Andrew Minor](https://mse.berkeley.edu/people_new/minor/) for teaching MATSCI 204: Materials Characterization and my advisor [Prof. Mark Asta](https://mse.berkeley.edu/people_new/asta/) for his unwavering encouragement for my education-related pursuits. 
This project is generously hosted on [GitHub](https://github.com/enze-chen/learning_modules) and [Google Colaboratory](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/XRD_trends.ipynb).